# Phase 13: Yield-Max + Opportunistic Shorts Backtest

Test the amplified yield and opportunistic short strategy:
- **Real USDT yield**: 6.5% avg APY (Kraken 6% + Bitrue 7%)
- **Opportunistic shorts**: RSI >70 spike in grind-down mode
- **Higher precision rewards**: 8.0x multiplier for whipsaw-free shorts
- **YieldManager integration**: Real rate simulation class
- Goal: Push toward -4% or better via yield + selective short PNL

In [ ]:
import sys
sys.path.insert(0, '../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from data_fetcher import DataFetcher
from portfolio import Portfolio
from orchestrator import RLOrchestrator
from risk_manager import RiskManager
from yield_manager import YieldManager
from strategies.dip_buy_lstm import generate_dip_signals, detect_dip_signal
from strategies.mean_reversion import calculate_vwap, is_above_vwap, get_vwap_deviation

print("Phase 13: Yield-Max + Opportunistic Shorts Backtest")
print("="*60)
print("Key Changes from Phase 12:")
print("  - YieldManager class for real rate simulation")
print("  - Avg USDT yield: 6.5% APY (Kraken 6% + Bitrue 7%)")
print("  - Opportunistic shorts: RSI >70 in grind-down mode")
print("  - Short precision: 8.0x for whipsaw-free shorts (up from 6.0x)")
print("  - Opportunistic VOL threshold: 4.5% ATR")
print("  - Target: -4% or better in bear markets")

## 1. Fetch Historical Data (Nov-Dec Bear Period)

In [ ]:
fetcher = DataFetcher()

# Fetch extended data to capture bear periods (Nov-Dec)
data = {}
for sym in ['XRP/USDT', 'BTC/USDT']:
    print(f"Fetching {sym}...")
    df = fetcher.fetch_ohlcv('kraken', sym, '1h', 2000)
    if not df.empty:
        data[sym] = df
        print(f"  {len(df)} candles from {df.index[0]} to {df.index[-1]}")

# Show price statistics
if 'XRP/USDT' in data:
    xrp = data['XRP/USDT']
    print(f"\nXRP range: ${xrp['close'].min():.4f} - ${xrp['close'].max():.4f}")
    print(f"XRP current: ${xrp['close'].iloc[-1]:.4f}")
    
if 'BTC/USDT' in data:
    btc = data['BTC/USDT']
    print(f"\nBTC range: ${btc['close'].min():.2f} - ${btc['close'].max():.2f}")
    print(f"BTC current: ${btc['close'].iloc[-1]:.2f}")

## 2. Phase 13: YieldManager Test

In [ ]:
# Test YieldManager class
ym = YieldManager()

print("YieldManager Configuration:")
print(f"  Kraken rate: {ym.kraken_rate*100:.1f}% APY")
print(f"  Bitrue rate: {ym.bitrue_rate*100:.1f}% APY")
print(f"  Average rate: {ym.avg_rate*100:.1f}% APY")

# Simulate yield accrual
test_balance = 1000.0
print(f"\nYield Projections for ${test_balance:.2f} USDT:")
print(f"  Daily: ${ym.simulate_monthly_yield(test_balance) / 30:.4f}")
print(f"  Monthly: ${ym.simulate_monthly_yield(test_balance):.2f}")
print(f"  Annual: ${ym.simulate_annual_yield(test_balance):.2f}")

# Test accrual
print(f"\nTest 4-hour accrual:")
yield_4h = ym.accrue(test_balance, hours=4)
print(f"  Yield: ${yield_4h:.6f}")
print(f"  Stats: {ym.get_stats()}")

## 3. Phase 13: Opportunistic Short Signal Analysis

In [ ]:
risk = RiskManager(max_drawdown=0.20, max_leverage=10.0)

def calculate_rsi(close, period=14):
    """Calculate RSI."""
    if len(close) < period + 1:
        return 50.0
    deltas = np.diff(close[-period-1:])
    gains = np.where(deltas > 0, deltas, 0)
    losses = np.where(deltas < 0, -deltas, 0)
    avg_gain = np.mean(gains) if len(gains) > 0 else 0
    avg_loss = np.mean(losses) if len(losses) > 0 else 0.001
    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs))

# Phase 13: Enhanced thresholds
RSI_RIP_THRESHOLD = 72  # Aggressive short
RSI_OPPORTUNISTIC = 70  # Phase 13: Opportunistic in grind-down
RSI_OVERBOUGHT = 65  # Selective short
ATR_BEAR = 0.04  # 4% ATR for bear mode
ATR_OPPORTUNISTIC = 0.045  # 4.5% ATR for opportunistic shorts

if 'XRP/USDT' in data:
    df = data['XRP/USDT']
    
    rip_shorts = []  # RSI > 72 + ATR > 4%
    opportunistic_shorts = []  # Phase 13: RSI > 70 + ATR > 4.5% in grind-down
    selective_shorts = []  # RSI 65-72 + ATR > 4%
    grind_down = []  # ATR > 4% but no short signal
    
    for i in range(50, len(df) - 24):
        close = df['close'].iloc[:i+1].values
        high = df['high'].iloc[:i+1].values
        low = df['low'].iloc[:i+1].values
        
        rsi = calculate_rsi(close)
        atr_pct = risk.calculate_atr_pct(high, low, close)
        
        # VWAP check
        slice_data = {'XRP/USDT': df.iloc[:i+1]}
        above_vwap = is_above_vwap(slice_data, 'XRP/USDT')
        
        # Future outcome (24h)
        future_prices = df['close'].iloc[i:i+24].values
        current_price = close[-1]
        min_future = np.min(future_prices)
        max_future = np.max(future_prices)
        drop_pct = (current_price - min_future) / current_price * 100
        bounce_pct = (max_future - current_price) / current_price * 100
        
        is_bear = atr_pct > ATR_BEAR
        is_opportunistic_vol = atr_pct > ATR_OPPORTUNISTIC
        
        if is_bear:
            entry = {
                'timestamp': df.index[i],
                'price': current_price,
                'rsi': rsi,
                'atr_pct': atr_pct * 100,
                'drop_pct': drop_pct,
                'bounce_pct': bounce_pct,
                'whipsaw_free': bounce_pct < 5,
                'above_vwap': above_vwap,
                'success': drop_pct > 3
            }
            
            if rsi > RSI_RIP_THRESHOLD and above_vwap:
                entry['type'] = 'rip_short'
                rip_shorts.append(entry)
            elif is_opportunistic_vol and rsi > RSI_OPPORTUNISTIC and above_vwap:
                # Phase 13: Opportunistic short in grind-down
                entry['type'] = 'opportunistic_short'
                opportunistic_shorts.append(entry)
            elif rsi > RSI_OVERBOUGHT and above_vwap:
                entry['type'] = 'selective_short'
                selective_shorts.append(entry)
            else:
                entry['type'] = 'grind_down'
                grind_down.append(entry)
    
    print("\nPhase 13 Short Signal Analysis:")
    print("="*60)
    
    rip_df = pd.DataFrame(rip_shorts)
    opp_df = pd.DataFrame(opportunistic_shorts)
    sel_df = pd.DataFrame(selective_shorts)
    grind_df = pd.DataFrame(grind_down)
    
    print(f"\n1. RIP SHORTS (RSI > 72 + ATR > 4% + above VWAP):")
    if len(rip_df) > 0:
        print(f"   Signals: {len(rip_df)}")
        print(f"   Success rate: {rip_df['success'].mean()*100:.1f}%")
        print(f"   Avg drop: {rip_df['drop_pct'].mean():.2f}%")
        print(f"   Whipsaw-free: {rip_df['whipsaw_free'].mean()*100:.1f}%")
    else:
        print(f"   Signals: 0")
    
    print(f"\n2. OPPORTUNISTIC SHORTS (RSI > 70 + ATR > 4.5% in grind-down):")
    if len(opp_df) > 0:
        print(f"   Signals: {len(opp_df)}")
        print(f"   Success rate: {opp_df['success'].mean()*100:.1f}%")
        print(f"   Avg drop: {opp_df['drop_pct'].mean():.2f}%")
        print(f"   Whipsaw-free: {opp_df['whipsaw_free'].mean()*100:.1f}%")
    else:
        print(f"   Signals: 0")
    
    print(f"\n3. SELECTIVE SHORTS (RSI 65-72 + ATR > 4% + above VWAP):")
    if len(sel_df) > 0:
        print(f"   Signals: {len(sel_df)}")
        print(f"   Success rate: {sel_df['success'].mean()*100:.1f}%")
        print(f"   Avg drop: {sel_df['drop_pct'].mean():.2f}%")
    else:
        print(f"   Signals: 0")
    
    print(f"\n4. GRIND-DOWN (ATR > 4% but no short signal = park + yield):")
    if len(grind_df) > 0:
        print(f"   Periods: {len(grind_df)}")
        print(f"   Avg ATR: {grind_df['atr_pct'].mean():.2f}%")
    else:
        print(f"   Periods: 0")

## 4. Phase 13: Enhanced Yield Simulation

In [ ]:
# Simulate USDT yield during grind-down and defensive periods
ym = YieldManager()

if 'grind_df' in dir() and len(grind_df) > 0:
    # Assume $800 USDT parked on average during grind-down
    avg_parked = 800
    total_grind_hours = len(grind_df) * 4  # Each period is ~4 hours
    
    # Use YieldManager for accurate yield
    total_yield = 0
    for _ in range(len(grind_df)):
        total_yield += ym.accrue(avg_parked, hours=4)
    
    stats = ym.get_stats()
    
    print(f"\nPhase 13 USDT Yield Simulation (YieldManager):")
    print(f"="*60)
    print(f"Kraken rate: {stats['kraken_rate']*100:.1f}%")
    print(f"Bitrue rate: {stats['bitrue_rate']*100:.1f}%")
    print(f"Average APY: {stats['avg_rate']*100:.1f}%")
    print(f"")
    print(f"Avg USDT parked: ${avg_parked:.2f}")
    print(f"Grind-down periods: {len(grind_df)}")
    print(f"Total grind hours: {total_grind_hours}")
    print(f"")
    print(f"Simulated yield: ${total_yield:.2f}")
    print(f"Accrual count: {stats['accrual_count']}")
    print(f"Avg per accrual: ${stats['avg_per_accrual']:.4f}")

## 5. Phase 13 Backtest - Yield-Max + Opportunistic Shorts

In [ ]:
# Initialize portfolio and orchestrator
starting = {'USDT': 1000.0, 'XRP': 500.0, 'BTC': 0.0}
portfolio = Portfolio(starting.copy())

try:
    orchestrator = RLOrchestrator(portfolio, data)
    rl_enabled = orchestrator.enabled
    print(f"\nRL Orchestrator: {'Enabled' if rl_enabled else 'Disabled'}")
    if rl_enabled:
        print(f"  Targets: {orchestrator.get_target_allocation()}")
        print(f"\nPhase 13 Thresholds:")
        print(f"  RSI rip threshold: {orchestrator.RSI_RIP_THRESHOLD}")
        print(f"  RSI opportunistic: {orchestrator.RSI_OPPORTUNISTIC}")
        print(f"  RSI overbought: {orchestrator.RSI_OVERBOUGHT}")
        print(f"  RSI short exit: {orchestrator.RSI_SHORT_EXIT}")
        print(f"  Bear vol threshold: {orchestrator.BEAR_VOL_THRESHOLD * 100}%")
        print(f"  Opportunistic vol: {orchestrator.OPPORTUNISTIC_VOL_THRESHOLD * 100}%")
        print(f"  Yield hours per step: {orchestrator.YIELD_HOURS_PER_STEP}")
except Exception as e:
    print(f"Could not initialize orchestrator: {e}")
    rl_enabled = False

In [ ]:
if rl_enabled and 'XRP/USDT' in data:
    df = data['XRP/USDT']
    btc_df = data.get('BTC/USDT')
    
    # Backtest loop
    results = []
    actions_log = []
    short_log = []
    yield_log = []
    
    step_size = 4  # Every 4 hours
    
    for i in range(100, len(df), step_size):
        timestamp = df.index[i]
        xrp_price = df['close'].iloc[i]
        btc_price = btc_df['close'].iloc[i] if btc_df is not None and i < len(btc_df) else 90000.0
        
        prices = {'XRP': xrp_price, 'BTC': btc_price, 'USDT': 1.0}
        
        # Update env step
        if orchestrator.env is not None:
            orchestrator.env.current_step = min(i, orchestrator.env.max_steps - 1)
        
        # Get RL decision
        result = orchestrator.decide_and_execute(prices)
        orchestrator.check_and_manage_positions(prices)
        orchestrator.update_env_step()
        
        # Track yield from portfolio
        yield_stats = portfolio.get_yield_stats()
        
        # Log
        total_value = portfolio.get_total_usd(prices)
        results.append({
            'timestamp': timestamp,
            'xrp_price': xrp_price,
            'btc_price': btc_price,
            'portfolio_value': total_value,
            'usdt': portfolio.balances.get('USDT', 0),
            'xrp': portfolio.balances.get('XRP', 0),
            'btc': portfolio.balances.get('BTC', 0),
            'volatility': result.get('volatility', 0),
            'mode': result.get('mode', 'unknown'),
            'bear_type': result.get('bear_type', 'none'),
            'market_state': result.get('market_state', 'unknown'),
            'rsi_xrp': result.get('rsi', {}).get('XRP', 50),
            'rsi_btc': result.get('rsi', {}).get('BTC', 50),
            'yield_earned': result.get('yield_earned', 0),
            'total_yield': yield_stats.get('total_yield_earned', 0),
            'avg_apy': yield_stats.get('avg_rate', 0.065)
        })
        
        if result.get('yield_earned', 0) > 0:
            yield_log.append({
                'timestamp': timestamp,
                'yield': result['yield_earned'],
                'usdt_balance': portfolio.balances.get('USDT', 0),
                'apy': yield_stats.get('avg_rate', 0.065)
            })
        
        if result.get('executed'):
            action_entry = {
                'timestamp': timestamp,
                'action': f"{result['asset']} {result['action_type']}",
                'leverage': result.get('leverage', 0),
                'mode': result.get('mode', 'unknown'),
                'bear_type': result.get('bear_type', 'none'),
                'trigger': result.get('offensive_trigger', result.get('short', 'standard')),
                'price': xrp_price,
                'confidence': result.get('confidence', 0)
            }
            actions_log.append(action_entry)
            
            if result.get('short'):
                short_log.append({
                    'timestamp': timestamp,
                    'asset': result['short'],
                    'entry_price': xrp_price if result['short'] == 'XRP' else btc_price,
                    'leverage': result.get('leverage', 0),
                    'collateral': result.get('collateral', 0),
                    'bear_type': result.get('bear_type', 'selective')
                })
    
    results_df = pd.DataFrame(results)
    results_df.set_index('timestamp', inplace=True)
    
    total_yield = portfolio.total_yield_earned
    
    print(f"\nBacktest complete: {len(results)} data points")
    print(f"Actions executed: {len(actions_log)}")
    print(f"Short positions opened: {len(short_log)}")
    print(f"Yield transactions: {len(yield_log)}")
    print(f"Total USDT yield earned: ${total_yield:.4f}")

In [ ]:
if 'results_df' in dir() and len(results_df) > 0:
    # Plot results
    fig, axes = plt.subplots(6, 1, figsize=(14, 20))
    
    # Portfolio value
    axes[0].plot(results_df.index, results_df['portfolio_value'], label='Portfolio Value', linewidth=2)
    axes[0].set_title('Phase 13: Portfolio Value - Yield-Max + Opportunistic Shorts')
    axes[0].set_ylabel('Value ($)')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Holdings
    axes[1].plot(results_df.index, results_df['usdt'], label='USDT', color='green')
    axes[1].plot(results_df.index, results_df['xrp'] * results_df['xrp_price'], label='XRP (USD)', color='blue')
    axes[1].plot(results_df.index, results_df['btc'] * results_df['btc_price'], label='BTC (USD)', color='orange')
    axes[1].set_title('Holdings (USD Value)')
    axes[1].set_ylabel('Value ($)')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    # Trading mode with bear types
    mode_colors = results_df['mode'].map({'offensive': 'green', 'defensive': 'orange', 'bear': 'red', 'unknown': 'gray'})
    mode_vals = results_df['mode'].map({'offensive': 1, 'defensive': 0, 'bear': -1, 'unknown': 0})
    axes[2].bar(results_df.index, mode_vals, color=mode_colors.values, alpha=0.6, width=0.02)
    axes[2].set_title('Trading Mode (1=Offensive, 0=Defensive/Grind, -1=Bear Short)')
    axes[2].set_ylabel('Mode')
    axes[2].set_ylim(-1.5, 1.5)
    
    # RSI with Phase 13 thresholds (72/70/65/40)
    axes[3].plot(results_df.index, results_df['rsi_xrp'], label='XRP RSI', color='blue')
    axes[3].plot(results_df.index, results_df['rsi_btc'], label='BTC RSI', color='orange')
    axes[3].axhline(y=30, color='green', linestyle='--', label='Oversold (Long)')
    axes[3].axhline(y=40, color='cyan', linestyle=':', label='Short Exit (RSI<40)')
    axes[3].axhline(y=65, color='yellow', linestyle='--', label='Overbought (Selective)')
    axes[3].axhline(y=70, color='orange', linestyle='-', linewidth=1.5, label='Opportunistic (RSI>70)')
    axes[3].axhline(y=72, color='red', linestyle='-', linewidth=2, label='Rip Short (RSI>72)')
    axes[3].fill_between(results_df.index, 0, 30, alpha=0.2, color='green')
    axes[3].fill_between(results_df.index, 70, 72, alpha=0.2, color='orange')
    axes[3].fill_between(results_df.index, 72, 100, alpha=0.3, color='red')
    axes[3].set_title('RSI Indicators (Phase 13: >72=Rip, >70=Opportunistic, >65=Selective)')
    axes[3].set_ylabel('RSI')
    axes[3].legend(loc='upper right', fontsize=8)
    axes[3].grid(True, alpha=0.3)
    
    # Volatility
    axes[4].plot(results_df.index, results_df['volatility'] * 100, label='ATR %', color='purple')
    axes[4].axhline(y=2, color='green', linestyle='--', label='Greed (<2%)')
    axes[4].axhline(y=4, color='orange', linestyle='--', label='Bear (>4%)')
    axes[4].axhline(y=4.5, color='red', linestyle='-', label='Opportunistic (>4.5%)')
    axes[4].fill_between(results_df.index, 4.5, results_df['volatility'].max() * 100 + 1, alpha=0.2, color='red')
    axes[4].set_title('Volatility (ATR %) - Phase 13: >4.5% = Opportunistic Short Eligible')
    axes[4].set_ylabel('ATR %')
    axes[4].legend()
    axes[4].grid(True, alpha=0.3)
    
    # Cumulative yield
    axes[5].plot(results_df.index, results_df['total_yield'], label='Cumulative USDT Yield', color='green', linewidth=2)
    axes[5].fill_between(results_df.index, 0, results_df['total_yield'], alpha=0.3, color='green')
    axes[5].set_title(f'Phase 13: Cumulative USDT Yield (6.5% avg APY) = ${total_yield:.4f}')
    axes[5].set_ylabel('Yield ($)')
    axes[5].legend()
    axes[5].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Summary stats
    initial_value = results_df['portfolio_value'].iloc[0]
    final_value = results_df['portfolio_value'].iloc[-1]
    max_value = results_df['portfolio_value'].max()
    min_value = results_df['portfolio_value'].min()
    pnl = final_value - initial_value
    roi = (pnl / initial_value) * 100
    max_dd = (max_value - min_value) / max_value * 100
    
    print(f"\n" + "="*60)
    print(f"PHASE 13 YIELD-MAX + OPPORTUNISTIC SHORTS RESULTS")
    print(f"="*60)
    print(f"Initial Value:     ${initial_value:.2f}")
    print(f"Final Value:       ${final_value:.2f}")
    print(f"USDT Yield:        ${total_yield:.4f}")
    print(f"")
    print(f"Total P&L:         ${pnl:+.2f} ({roi:+.1f}%)")
    print(f"Max Value:         ${max_value:.2f}")
    print(f"Min Value:         ${min_value:.2f}")
    print(f"Max Drawdown:      {max_dd:.1f}%")
    print(f"")
    print(f"Time in Offensive (Long): {(results_df['mode'] == 'offensive').mean()*100:.1f}%")
    print(f"Time in Bear (Short):     {(results_df['mode'] == 'bear').mean()*100:.1f}%")
    print(f"Time in Defensive:        {(results_df['mode'] == 'defensive').mean()*100:.1f}%")
    print(f"")
    print(f"Bear Type Distribution:")
    print(results_df['bear_type'].value_counts())

## 6. Short Position Analysis (Including Opportunistic)

In [ ]:
if 'short_log' in dir() and len(short_log) > 0:
    short_df = pd.DataFrame(short_log)
    
    print("\nShort Position Analysis:")
    print("="*60)
    print(f"Total shorts opened: {len(short_df)}")
    
    print(f"\nBy Bear Type:")
    print(short_df['bear_type'].value_counts())
    
    print(f"\nBy Asset:")
    print(short_df['asset'].value_counts())
    
    print(f"\nAvg Leverage: {short_df['leverage'].mean():.1f}x")
    print(f"Avg Collateral: ${short_df['collateral'].mean():.2f}")
    print(f"Total Collateral Deployed: ${short_df['collateral'].sum():.2f}")
    
    print(f"\nShort Details:")
    print(short_df.to_string())
else:
    print("\nNo short positions opened during backtest")

## 7. Benchmark Comparison (Phase 11 vs 12 vs 13)

In [ ]:
if 'XRP/USDT' in data and 'results_df' in dir():
    df = data['XRP/USDT']
    
    # Buy and hold benchmarks
    initial_xrp = df['close'].iloc[100]
    final_xrp = df['close'].iloc[-1]
    xrp_return = (final_xrp - initial_xrp) / initial_xrp * 100
    
    if 'BTC/USDT' in data:
        btc_df = data['BTC/USDT']
        initial_btc = btc_df['close'].iloc[100]
        final_btc = btc_df['close'].iloc[-1]
        btc_return = (final_btc - initial_btc) / initial_btc * 100
    else:
        btc_return = 0
    
    # Our strategy return
    strategy_return = roi if 'roi' in dir() else 0
    
    # Previous phase references
    phase10_return = -8.6  # From Phase 10
    phase11_return = -6.3  # From Phase 11
    phase12_return = -7.5  # From Phase 12
    
    print(f"\n" + "="*60)
    print(f"BENCHMARK COMPARISON")
    print(f"="*60)
    print(f"XRP Buy & Hold:     {xrp_return:+.1f}%")
    print(f"BTC Buy & Hold:     {btc_return:+.1f}%")
    print(f"")
    print(f"Phase 10 Strategy:  {phase10_return:+.1f}%")
    print(f"Phase 11 Strategy:  {phase11_return:+.1f}%")
    print(f"Phase 12 Strategy:  {phase12_return:+.1f}%")
    print(f"Phase 13 Strategy:  {strategy_return:+.1f}% (with YieldManager + opportunistic shorts)")
    print(f"")
    print(f"Phase 13 vs Phase 12: {strategy_return - phase12_return:+.1f}%")
    print(f"Phase 13 vs Phase 11: {strategy_return - phase11_return:+.1f}%")
    print(f"Phase 13 vs XRP:      {strategy_return - xrp_return:+.1f}%")
    print(f"Phase 13 vs BTC:      {strategy_return - btc_return:+.1f}%")
    
    # Performance assessment
    if strategy_return >= -4:
        print(f"\n[GOAL MET] Target -4% or better achieved!")
    elif strategy_return >= 0:
        print(f"\n[EXCELLENT] Bear profit flip achieved! Positive return in bear market!")
    elif strategy_return > phase12_return:
        improvement = strategy_return - phase12_return
        print(f"\n[PROGRESS] Phase 13 improved by {improvement:+.1f}% over Phase 12")
    else:
        print(f"\n[REVIEW] Strategy needs further tuning")

## 8. Phase 13 Short P&L Simulation (8.0x Precision)

In [ ]:
# Simulate potential short profits with Phase 13 precision shorts
all_shorts = []
if 'rip_df' in dir() and len(rip_df) > 0:
    all_shorts.extend(rip_df.to_dict('records'))
if 'opp_df' in dir() and len(opp_df) > 0:
    all_shorts.extend(opp_df.to_dict('records'))
if 'sel_df' in dir() and len(sel_df) > 0:
    all_shorts.extend(sel_df.to_dict('records'))

if len(all_shorts) > 0:
    print("\nPhase 13 Short P&L Simulation (8.0x Precision):")
    print("="*60)
    
    total_pnl = 0
    rip_pnl = 0
    opp_pnl = 0
    sel_pnl = 0
    whipsaw_free_wins = 0
    whipsaw_losses = 0
    
    for short in all_shorts:
        short_type = short.get('type', 'selective_short')
        is_rip = short_type == 'rip_short'
        is_opp = short_type == 'opportunistic_short'
        
        # Collateral by type
        if is_rip:
            collateral = 1000 * 0.10  # 10% for rip
            leverage = 5
        elif is_opp:
            collateral = 1000 * 0.04  # 4% for opportunistic
            leverage = 4
        else:
            collateral = 1000 * 0.06  # 6% for selective
            leverage = 4
        
        # P&L = collateral * leverage * drop_pct
        pnl = collateral * leverage * (short['drop_pct'] / 100)
        
        # Phase 13: Track whipsaw-free
        if short.get('whipsaw_free', False) and pnl > 0:
            whipsaw_free_wins += 1
        elif pnl <= 0:
            whipsaw_losses += 1
        
        total_pnl += pnl
        if is_rip:
            rip_pnl += pnl
        elif is_opp:
            opp_pnl += pnl
        else:
            sel_pnl += pnl
    
    rip_count = len([s for s in all_shorts if s.get('type') == 'rip_short'])
    opp_count = len([s for s in all_shorts if s.get('type') == 'opportunistic_short'])
    sel_count = len(all_shorts) - rip_count - opp_count
    
    print(f"\nRIP SHORTS (RSI > 72):")
    print(f"  Count: {rip_count}")
    print(f"  P&L: ${rip_pnl:+.2f}")
    if rip_count > 0:
        print(f"  Avg P&L: ${rip_pnl/rip_count:+.2f}")
    
    print(f"\nOPPORTUNISTIC SHORTS (RSI > 70 + ATR > 4.5%):")
    print(f"  Count: {opp_count}")
    print(f"  P&L: ${opp_pnl:+.2f}")
    if opp_count > 0:
        print(f"  Avg P&L: ${opp_pnl/opp_count:+.2f}")
    
    print(f"\nSELECTIVE SHORTS (RSI 65-70):")
    print(f"  Count: {sel_count}")
    print(f"  P&L: ${sel_pnl:+.2f}")
    if sel_count > 0:
        print(f"  Avg P&L: ${sel_pnl/sel_count:+.2f}")
    
    print(f"\nOVERALL:")
    print(f"  Total trades: {len(all_shorts)}")
    print(f"  Whipsaw-free wins: {whipsaw_free_wins}")
    print(f"  Whipsaw losses: {whipsaw_losses}")
    print(f"  Total P&L: ${total_pnl:+.2f}")
    
    # Combined impact
    phase12_base = -150.0  # Approximate from Phase 12
    combined_pnl = phase12_base + total_pnl + total_yield
    print(f"\nCOMBINED IMPACT:")
    print(f"  Phase 12 base P&L: ${phase12_base:+.2f}")
    print(f"  + Short profits: ${total_pnl:+.2f}")
    print(f"  + USDT yield: ${total_yield:+.4f}")
    print(f"  = Phase 13 P&L: ${combined_pnl:+.2f}")
    
    if combined_pnl >= -40:
        print(f"\n[SUCCESS] Phase 13 target (-4% or better) achievable!")
    elif combined_pnl > phase12_base:
        print(f"\n[PROGRESS] Reduced losses by ${combined_pnl - phase12_base:.2f}")
else:
    print("\nNo short signals detected")

## 9. Next Steps

1. **Retrain PPO**: Run 2M timesteps with Phase 13 rewards (8.0x precision, opportunistic shorts)
2. **Live Paper**: Start `python src/live_paper.py --interval 10`
3. **Monitor**: Check logs for opportunistic short triggers and yield accrual
4. **Tune Further**: If still >-4%, try RSI >68 for opportunistic, lower ATR threshold to 4%

In [ ]:
print("\n" + "="*60)
print("PHASE 13 BACKTEST COMPLETE")
print("="*60)
print("")
print("To retrain with Phase 13 rewards:")
print("  python src/main.py --mode train-rl --timesteps 2000000 --device cuda")
print("")
print("To start live paper trading (10 min intervals):")
print("  python src/live_paper.py --interval 10")
print("")
print("Phase 13 Yield-Max + Opportunistic Short thresholds:")
print(f"  Rip short: RSI > 72 + ATR > 4% + above VWAP")
print(f"  Opportunistic short: RSI > 70 + ATR > 4.5% in grind-down")
print(f"  Selective short: RSI > 65 + ATR > 4% + above VWAP")
print(f"  Grind-down: ATR > 4% but no short signal = park + yield")
print(f"  RSI exit: Auto-close shorts when RSI < 40")
print(f"  Real USDT yield: 6.5% avg APY (YieldManager)")
print(f"  Short precision: 8.0x for whipsaw-free shorts")
print(f"  Decay penalty: 0.3")